In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window

In [3]:
spark = SparkSession.builder.appName("PySparkPrac").getOrCreate()

25/08/26 11:17:16 WARN Utils: Your hostname, neosoft-Latitude-5420 resolves to a loopback address: 127.0.1.1; using 10.0.61.246 instead (on interface wlp0s20f3)
25/08/26 11:17:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/26 11:17:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data = [
    ("U1", "2025-01-01"),
    ("U1", "2025-01-05"),
    ("U1", "2025-01-10"),
    ("U2", "2025-01-02"),
    ("U2", "2025-01-04")
]

columns = ['user_id','session_date']
df = spark.createDataFrame(data, columns)

In [5]:
df = df.withColumn("session_date", to_date("session_date"))

In [6]:
df.show()

+-------+------------+
|user_id|session_date|
+-------+------------+
|     U1|  2025-01-01|
|     U1|  2025-01-05|
|     U1|  2025-01-10|
|     U2|  2025-01-02|
|     U2|  2025-01-04|
+-------+------------+



In [8]:
w = Window.partitionBy("user_id").orderBy("session_date")

lag_df = df.withColumn("prev_session_date", lag("session_date").over(w))

In [9]:
lag_df.show()

+-------+------------+-----------------+
|user_id|session_date|prev_session_date|
+-------+------------+-----------------+
|     U1|  2025-01-01|             NULL|
|     U1|  2025-01-05|       2025-01-01|
|     U1|  2025-01-10|       2025-01-05|
|     U2|  2025-01-02|             NULL|
|     U2|  2025-01-04|       2025-01-02|
+-------+------------+-----------------+



In [12]:
session_gaps = lag_df.withColumn("days_since_last_session", date_diff("session_date","prev_session_date")).drop("prev_session_date")

In [13]:
session_gaps.show()

+-------+------------+-----------------------+
|user_id|session_date|days_since_last_session|
+-------+------------+-----------------------+
|     U1|  2025-01-01|                   NULL|
|     U1|  2025-01-05|                      4|
|     U1|  2025-01-10|                      5|
|     U2|  2025-01-02|                   NULL|
|     U2|  2025-01-04|                      2|
+-------+------------+-----------------------+



In [16]:
data = [
    ("C1", "2025-01-01", 200),
("C1", "2025-01-10", 300),
("C2", "2025-01-05", 150),
("C3", "2025-01-02", 100),
("C3", "2025-01-04", 200),
]

columns = ["customer_id", "purchase_date", "amount"]
purchases = spark.createDataFrame(data, columns)

In [18]:
purchases = purchases.withColumn("purchase_date", to_date("purchase_date"))
purchases.createOrReplaceTempView("purchases")


In [22]:
result = spark.sql('''
select customer_id from purchases group by customer_id having count(*) = 1
''')

In [23]:
result.show()

+-----------+
|customer_id|
+-----------+
|         C2|
+-----------+

